In [1]:
import numpy as np
from numpy.linalg import norm
import pickle
import matplotlib.pyplot as plt
import copy
import math
import time
import scipy
from pathlib import Path
import scipy.io
import cmath

import cvxpy as cp

%matplotlib inline

# EG: SDP for worst case $\|F(x^{k+1})\|^2 - \|F(x^k)\|^2$

In fact, we study here slightly different method:
$$
x^{k+1} = x^k - \gamma F\left(x^k - \gamma F(x^k)\right).
$$

### Define parameters and the matrices for the problem

$\gamma = \frac{1}{5L}$

In [2]:
# Lipschitz parameter and stepsize
L = 1.0
gamma1 = 1.0 / (5*L)
gamma2 = gamma1

# Matrices for SDP
M0 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, -1, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 1]])

M1 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, 1, -1.0/2, 0],
                   [0, -1.0/2, 0, 0],
                   [0, 0, 0, 0]])

M2 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, (L**2)*(gamma1**2)-1, 1, 0],
                   [0, 1, -1, 0],
                   [0, 0, 0, 0]])

M3 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, 0, 1.0/2, 0],
                   [0, 1.0/2, 0, -1.0/2],
                   [0, 0, -1.0/2, 0]])

M4 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, -1, 0, 1],
                   [0, 0, (L**2)*(gamma2**2), 0],
                   [0, 1, 0, -1]])

M5 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, 0, -gamma1/2, gamma1/2],
                   [0, -gamma1/2, gamma2, -gamma2/2],
                   [0, gamma1/2, -gamma2/2, 0]]) / gamma1

M6 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, (L**2)*(gamma1**2), -(L**2)*(gamma1*gamma2), 0],
                   [0, -(L**2)*(gamma1*gamma2), (L**2)*(gamma2**2)-1, 1],
                   [0, 0, 1, -1]])

M7 = 1.0*np.array([[0, 0.5, 0, 0], 
                   [0.5, 0, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

M8 = 1.0*np.array([[L**2, 0, 0, 0], 
                   [0, -1, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

M9 = 1.0*np.array([[0, 0, 0.5, 0], 
                   [0, 0, -gamma1/2, 0],
                   [0.5, -gamma1/2, 0, 0],
                   [0, 0, 0, 0]])

M10 = 1.0*np.array([[L**2, -(L**2)*gamma1, 0, 0], 
                   [-(L**2)*gamma1, (L**2)*(gamma1**2), 0, 0],
                   [0, 0, -1, 0],
                   [0, 0, 0, 0]])

M11 = 1.0*np.array([[0, 0, 0, 0.5], 
                   [0, 0, 0, 0],
                   [0, 0, 0, -gamma2/2],
                   [0.5, 0, -gamma2/2, 0]])

M12 = 1.0*np.array([[(L**2), 0, -(L**2)*gamma2, 0], 
                   [0, 0, 0, 0],
                   [-(L**2)*gamma2, 0, (L**2)*(gamma2**2), 0],
                   [0, 0, 0, -1]])

M13 = 1.0*np.array([[1, 0, 0, 0], 
                   [0, 0, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

### Define and solve SDP problem

In [3]:
%%time
G = cp.Variable((4,4), symmetric=True)

constraints = [G >> 0]
constraints += [cp.trace(M1 @ G) >= 0]
constraints += [cp.trace(M2 @ G) >= 0]
constraints += [cp.trace(M3 @ G) >= 0]
constraints += [cp.trace(M4 @ G) >= 0]
constraints += [cp.trace(M5 @ G) >= 0]
constraints += [cp.trace(M6 @ G) >= 0]
constraints += [cp.trace(M7 @ G) >= 0]
constraints += [cp.trace(M8 @ G) >= 0]
constraints += [cp.trace(M9 @ G) >= 0]
constraints += [cp.trace(M10 @ G) >= 0]
constraints += [cp.trace(M11 @ G) >= 0]
constraints += [cp.trace(M12 @ G) >= 0]
constraints += [cp.trace(M13 @ G) == 1]

prob = cp.Problem(cp.Maximize(cp.trace(M0 @ G)),
                  constraints)

prob.solve()

Wall time: 128 ms


1.431010310248837e-10

In [4]:
print("The optimal value is", prob.value)
print("G = ")
print(G.value)

The optimal value is 1.431010310248837e-10
G = 
[[1.         0.07927615 0.07927615 0.07927615]
 [0.07927615 0.30908499 0.30908499 0.30908499]
 [0.07927615 0.30908499 0.30908499 0.30908499]
 [0.07927615 0.30908499 0.30908499 0.30908499]]


### Print dual variables

In [5]:
print("Dual variable 1 : ", constraints[1].dual_value)
print("Dual variable 2 : ", constraints[2].dual_value)
print("Dual variable 3 : ", constraints[3].dual_value)
print("Dual variable 4 : ", constraints[4].dual_value)
print("Dual variable 5 : ", constraints[5].dual_value)
print("Dual variable 6 : ", constraints[6].dual_value)
print("Dual variable 7 : ", constraints[7].dual_value)
print("Dual variable 8 : ", constraints[8].dual_value)
print("Dual variable 9 : ", constraints[9].dual_value)
print("Dual variable 10: ", constraints[10].dual_value)
print("Dual variable 11: ", constraints[11].dual_value)
print("Dual variable 12: ", constraints[12].dual_value)
print("Dual variable 13: ", constraints[13].dual_value)

Dual variable 1 :  4.77030987477042e-10
Dual variable 2 :  0.0
Dual variable 3 :  1.999999999801979
Dual variable 4 :  0.0
Dual variable 5 :  0.5803686163130072
Dual variable 6 :  1.337493848880875
Dual variable 7 :  0.0
Dual variable 8 :  0.0
Dual variable 9 :  0.0
Dual variable 10:  0.0
Dual variable 11:  0.0
Dual variable 12:  0.0
Dual variable 13:  1.6145668435654499e-10


### Define parameters and the matrices for the problem

$\gamma = \frac{1}{4L}$

In [6]:
# Lipschitz parameter and stepsize
L = 1.0
gamma1 = 1.0 / (4*L)
gamma2 = gamma1

# Matrices for SDP
M0 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, -1, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 1]])

M1 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, 1, -1.0/2, 0],
                   [0, -1.0/2, 0, 0],
                   [0, 0, 0, 0]])

M2 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, (L**2)*(gamma1**2)-1, 1, 0],
                   [0, 1, -1, 0],
                   [0, 0, 0, 0]])

M3 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, 0, 1.0/2, 0],
                   [0, 1.0/2, 0, -1.0/2],
                   [0, 0, -1.0/2, 0]])

M4 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, -1, 0, 1],
                   [0, 0, (L**2)*(gamma2**2), 0],
                   [0, 1, 0, -1]])

M5 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, 0, -gamma1/2, gamma1/2],
                   [0, -gamma1/2, gamma2, -gamma2/2],
                   [0, gamma1/2, -gamma2/2, 0]]) / gamma1

M6 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, (L**2)*(gamma1**2), -(L**2)*(gamma1*gamma2), 0],
                   [0, -(L**2)*(gamma1*gamma2), (L**2)*(gamma2**2)-1, 1],
                   [0, 0, 1, -1]])

M7 = 1.0*np.array([[0, 0.5, 0, 0], 
                   [0.5, 0, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

M8 = 1.0*np.array([[L**2, 0, 0, 0], 
                   [0, -1, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

M9 = 1.0*np.array([[0, 0, 0.5, 0], 
                   [0, 0, -gamma1/2, 0],
                   [0.5, -gamma1/2, 0, 0],
                   [0, 0, 0, 0]])

M10 = 1.0*np.array([[L**2, -(L**2)*gamma1, 0, 0], 
                   [-(L**2)*gamma1, (L**2)*(gamma1**2), 0, 0],
                   [0, 0, -1, 0],
                   [0, 0, 0, 0]])

M11 = 1.0*np.array([[0, 0, 0, 0.5], 
                   [0, 0, 0, 0],
                   [0, 0, 0, -gamma2/2],
                   [0.5, 0, -gamma2/2, 0]])

M12 = 1.0*np.array([[(L**2), 0, -(L**2)*gamma2, 0], 
                   [0, 0, 0, 0],
                   [-(L**2)*gamma2, 0, (L**2)*(gamma2**2), 0],
                   [0, 0, 0, -1]])

M13 = 1.0*np.array([[1, 0, 0, 0], 
                   [0, 0, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

### Define and solve SDP problem

In [7]:
%%time
G = cp.Variable((4,4), symmetric=True)

constraints = [G >> 0]
constraints += [cp.trace(M1 @ G) >= 0]
constraints += [cp.trace(M2 @ G) >= 0]
constraints += [cp.trace(M3 @ G) >= 0]
constraints += [cp.trace(M4 @ G) >= 0]
constraints += [cp.trace(M5 @ G) >= 0]
constraints += [cp.trace(M6 @ G) >= 0]
constraints += [cp.trace(M7 @ G) >= 0]
constraints += [cp.trace(M8 @ G) >= 0]
constraints += [cp.trace(M9 @ G) >= 0]
constraints += [cp.trace(M10 @ G) >= 0]
constraints += [cp.trace(M11 @ G) >= 0]
constraints += [cp.trace(M12 @ G) >= 0]
constraints += [cp.trace(M13 @ G) == 1]

prob = cp.Problem(cp.Maximize(cp.trace(M0 @ G)),
                  constraints)

prob.solve()

Wall time: 30 ms


-1.1110082932697107e-05

In [8]:
print("The optimal value is", prob.value)
print("G = ")
print(G.value)

The optimal value is -1.1110082932697107e-05
G = 
[[1.0000053  0.09827903 0.09827698 0.09829571]
 [0.09827903 0.30086963 0.30084578 0.30086077]
 [0.09827698 0.30084578 0.30082181 0.30083944]
 [0.09829571 0.30086077 0.30083944 0.30085852]]


### Print dual variables

In [9]:
print("Dual variable 1 : ", constraints[1].dual_value)
print("Dual variable 2 : ", constraints[2].dual_value)
print("Dual variable 3 : ", constraints[3].dual_value)
print("Dual variable 4 : ", constraints[4].dual_value)
print("Dual variable 5 : ", constraints[5].dual_value)
print("Dual variable 6 : ", constraints[6].dual_value)
print("Dual variable 7 : ", constraints[7].dual_value)
print("Dual variable 8 : ", constraints[8].dual_value)
print("Dual variable 9 : ", constraints[9].dual_value)
print("Dual variable 10: ", constraints[10].dual_value)
print("Dual variable 11: ", constraints[11].dual_value)
print("Dual variable 12: ", constraints[12].dual_value)
print("Dual variable 13: ", constraints[13].dual_value)

Dual variable 1 :  4.001109602667928e-05
Dual variable 2 :  0.0
Dual variable 3 :  2.0000650018484327
Dual variable 4 :  0.0
Dual variable 5 :  0.5859157618454286
Dual variable 6 :  1.331117064465093
Dual variable 7 :  0.0
Dual variable 8 :  0.0
Dual variable 9 :  0.0
Dual variable 10:  0.0
Dual variable 11:  0.0
Dual variable 12:  0.0
Dual variable 13:  -2.0038238481393826e-05


### Define parameters and the matrices for the problem

$\gamma = \frac{1}{3L}$

In [10]:
# Lipschitz parameter and stepsize
L = 1.0
gamma1 = 1.0 / (3*L)
gamma2 = gamma1

# Matrices for SDP
M0 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, -1, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 1]])

M1 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, 1, -1.0/2, 0],
                   [0, -1.0/2, 0, 0],
                   [0, 0, 0, 0]])

M2 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, (L**2)*(gamma1**2)-1, 1, 0],
                   [0, 1, -1, 0],
                   [0, 0, 0, 0]])

M3 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, 0, 1.0/2, 0],
                   [0, 1.0/2, 0, -1.0/2],
                   [0, 0, -1.0/2, 0]])

M4 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, -1, 0, 1],
                   [0, 0, (L**2)*(gamma2**2), 0],
                   [0, 1, 0, -1]])

M5 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, 0, -gamma1/2, gamma1/2],
                   [0, -gamma1/2, gamma2, -gamma2/2],
                   [0, gamma1/2, -gamma2/2, 0]]) / gamma1

M6 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, (L**2)*(gamma1**2), -(L**2)*(gamma1*gamma2), 0],
                   [0, -(L**2)*(gamma1*gamma2), (L**2)*(gamma2**2)-1, 1],
                   [0, 0, 1, -1]])

M7 = 1.0*np.array([[0, 0.5, 0, 0], 
                   [0.5, 0, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

M8 = 1.0*np.array([[L**2, 0, 0, 0], 
                   [0, -1, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

M9 = 1.0*np.array([[0, 0, 0.5, 0], 
                   [0, 0, -gamma1/2, 0],
                   [0.5, -gamma1/2, 0, 0],
                   [0, 0, 0, 0]])

M10 = 1.0*np.array([[L**2, -(L**2)*gamma1, 0, 0], 
                   [-(L**2)*gamma1, (L**2)*(gamma1**2), 0, 0],
                   [0, 0, -1, 0],
                   [0, 0, 0, 0]])

M11 = 1.0*np.array([[0, 0, 0, 0.5], 
                   [0, 0, 0, 0],
                   [0, 0, 0, -gamma2/2],
                   [0.5, 0, -gamma2/2, 0]])

M12 = 1.0*np.array([[(L**2), 0, -(L**2)*gamma2, 0], 
                   [0, 0, 0, 0],
                   [-(L**2)*gamma2, 0, (L**2)*(gamma2**2), 0],
                   [0, 0, 0, -1]])

M13 = 1.0*np.array([[1, 0, 0, 0], 
                   [0, 0, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

### Define and solve SDP problem

In [11]:
%%time
G = cp.Variable((4,4), symmetric=True)

constraints = [G >> 0]
constraints += [cp.trace(M1 @ G) >= 0]
constraints += [cp.trace(M2 @ G) >= 0]
constraints += [cp.trace(M3 @ G) >= 0]
constraints += [cp.trace(M4 @ G) >= 0]
constraints += [cp.trace(M5 @ G) >= 0]
constraints += [cp.trace(M6 @ G) >= 0]
constraints += [cp.trace(M7 @ G) >= 0]
constraints += [cp.trace(M8 @ G) >= 0]
constraints += [cp.trace(M9 @ G) >= 0]
constraints += [cp.trace(M10 @ G) >= 0]
constraints += [cp.trace(M11 @ G) >= 0]
constraints += [cp.trace(M12 @ G) >= 0]
constraints += [cp.trace(M13 @ G) == 1]

prob = cp.Problem(cp.Maximize(cp.trace(M0 @ G)),
                  constraints)

prob.solve()

Wall time: 29 ms


-2.887441768906207e-06

In [12]:
print("The optimal value is", prob.value)
print("G = ")
print(G.value)

The optimal value is -2.887441768906207e-06
G = 
[[1.00000046 0.12070312 0.12070496 0.12070714]
 [0.12070312 0.32381443 0.32381085 0.32381296]
 [0.12070496 0.32381085 0.32381018 0.32381151]
 [0.12070714 0.32381296 0.32381151 0.32381154]]


### Print dual variables

In [13]:
print("Dual variable 1 : ", constraints[1].dual_value)
print("Dual variable 2 : ", constraints[2].dual_value)
print("Dual variable 3 : ", constraints[3].dual_value)
print("Dual variable 4 : ", constraints[4].dual_value)
print("Dual variable 5 : ", constraints[5].dual_value)
print("Dual variable 6 : ", constraints[6].dual_value)
print("Dual variable 7 : ", constraints[7].dual_value)
print("Dual variable 8 : ", constraints[8].dual_value)
print("Dual variable 9 : ", constraints[9].dual_value)
print("Dual variable 10: ", constraints[10].dual_value)
print("Dual variable 11: ", constraints[11].dual_value)
print("Dual variable 12: ", constraints[12].dual_value)
print("Dual variable 13: ", constraints[13].dual_value)

Dual variable 1 :  2.755998537282654e-06
Dual variable 2 :  0.0
Dual variable 3 :  2.0000049348919617
Dual variable 4 :  0.0
Dual variable 5 :  0.43020469368251846
Dual variable 6 :  1.427219078631502
Dual variable 7 :  0.0
Dual variable 8 :  0.0
Dual variable 9 :  0.0
Dual variable 10:  0.0
Dual variable 11:  0.0
Dual variable 12:  0.0
Dual variable 13:  -2.66371289206901e-06


### Define parameters and the matrices for the problem

$\gamma = \frac{1}{2L}$

In [18]:
# Lipschitz parameter and stepsize
L = 1.0
gamma1 = 1.0 / (2*L)
gamma2 = gamma1

# Matrices for SDP
M0 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, -1, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 1]])

M1 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, 1, -1.0/2, 0],
                   [0, -1.0/2, 0, 0],
                   [0, 0, 0, 0]])

M2 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, (L**2)*(gamma1**2)-1, 1, 0],
                   [0, 1, -1, 0],
                   [0, 0, 0, 0]])

M3 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, 0, 1.0/2, 0],
                   [0, 1.0/2, 0, -1.0/2],
                   [0, 0, -1.0/2, 0]])

M4 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, -1, 0, 1],
                   [0, 0, (L**2)*(gamma2**2), 0],
                   [0, 1, 0, -1]])

M5 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, 0, -gamma1/2, gamma1/2],
                   [0, -gamma1/2, gamma2, -gamma2/2],
                   [0, gamma1/2, -gamma2/2, 0]]) / gamma1

M6 = 1.0*np.array([[0, 0, 0, 0], 
                   [0, (L**2)*(gamma1**2), -(L**2)*(gamma1*gamma2), 0],
                   [0, -(L**2)*(gamma1*gamma2), (L**2)*(gamma2**2)-1, 1],
                   [0, 0, 1, -1]])

M7 = 1.0*np.array([[0, 0.5, 0, 0], 
                   [0.5, 0, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

M8 = 1.0*np.array([[L**2, 0, 0, 0], 
                   [0, -1, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

M9 = 1.0*np.array([[0, 0, 0.5, 0], 
                   [0, 0, -gamma1/2, 0],
                   [0.5, -gamma1/2, 0, 0],
                   [0, 0, 0, 0]])

M10 = 1.0*np.array([[L**2, -(L**2)*gamma1, 0, 0], 
                   [-(L**2)*gamma1, (L**2)*(gamma1**2), 0, 0],
                   [0, 0, -1, 0],
                   [0, 0, 0, 0]])

M11 = 1.0*np.array([[0, 0, 0, 0.5], 
                   [0, 0, 0, 0],
                   [0, 0, 0, -gamma2/2],
                   [0.5, 0, -gamma2/2, 0]])

M12 = 1.0*np.array([[(L**2), 0, -(L**2)*gamma2, 0], 
                   [0, 0, 0, 0],
                   [-(L**2)*gamma2, 0, (L**2)*(gamma2**2), 0],
                   [0, 0, 0, -1]])

M13 = 1.0*np.array([[1, 0, 0, 0], 
                   [0, 0, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

### Define and solve SDP problem

In [19]:
%%time
G = cp.Variable((4,4), symmetric=True)

constraints = [G >> 0]
constraints += [cp.trace(M1 @ G) >= 0]
constraints += [cp.trace(M2 @ G) >= 0]
constraints += [cp.trace(M3 @ G) >= 0]
constraints += [cp.trace(M4 @ G) >= 0]
constraints += [cp.trace(M5 @ G) >= 0]
constraints += [cp.trace(M6 @ G) >= 0]
constraints += [cp.trace(M7 @ G) >= 0]
constraints += [cp.trace(M8 @ G) >= 0]
constraints += [cp.trace(M9 @ G) >= 0]
constraints += [cp.trace(M10 @ G) >= 0]
constraints += [cp.trace(M11 @ G) >= 0]
constraints += [cp.trace(M12 @ G) >= 0]
constraints += [cp.trace(M13 @ G) == 1]

prob = cp.Problem(cp.Maximize(cp.trace(M0 @ G)),
                  constraints)

prob.solve()

Wall time: 28 ms


-2.2439360946036047e-06

In [20]:
print("The optimal value is", prob.value)
print("G = ")
print(G.value)

The optimal value is -2.2439360946036047e-06
G = 
[[1.00000069 0.16229743 0.16230239 0.16229271]
 [0.16229743 0.29713577 0.29713687 0.29713668]
 [0.16230239 0.29713687 0.29713931 0.29713707]
 [0.16229271 0.29713668 0.29713707 0.29713353]]


### Print dual variables

In [21]:
print("Dual variable 1 : ", constraints[1].dual_value)
print("Dual variable 2 : ", constraints[2].dual_value)
print("Dual variable 3 : ", constraints[3].dual_value)
print("Dual variable 4 : ", constraints[4].dual_value)
print("Dual variable 5 : ", constraints[5].dual_value)
print("Dual variable 6 : ", constraints[6].dual_value)
print("Dual variable 7 : ", constraints[7].dual_value)
print("Dual variable 8 : ", constraints[8].dual_value)
print("Dual variable 9 : ", constraints[9].dual_value)
print("Dual variable 10: ", constraints[10].dual_value)
print("Dual variable 11: ", constraints[11].dual_value)
print("Dual variable 12: ", constraints[12].dual_value)
print("Dual variable 13: ", constraints[13].dual_value)

Dual variable 1 :  0.0
Dual variable 2 :  0.0
Dual variable 3 :  2.0000119980201756
Dual variable 4 :  0.0
Dual variable 5 :  0.4212982062584506
Dual variable 6 :  1.3436820772438518
Dual variable 7 :  0.0
Dual variable 8 :  0.0
Dual variable 9 :  0.0
Dual variable 10:  0.0
Dual variable 11:  0.0
Dual variable 12:  0.0
Dual variable 13:  -2.320994038404713e-06


### We see that dual variables are almost independent of $\gamma$. Therefore, we make a following guess:

$$\lambda_3 = 2, \quad \lambda_5 = \frac{1}{2},\quad \lambda_6 = \frac{3}{2}$$

### and other dual variables are zeros